# Introduction to Iris

* from https://scitools.org.uk/iris/docs/latest/ : "Iris implements a data model based on the CF conventions giving you a powerful, format-agnostic interface for working with your data. It excels when working with multi-dimensional Earth Science data, where tabular representations become unwieldy and inefficient. "
* **Iris** is useful for using with Netcdf files, if they are written with CF conventions in mind. 
* The MetOffice uses Iris with their fieldsfiles (.pp)


First let's do some importing

In [1]:
import iris # import the Iris module

In [ ]:
import matplotlib.pyplot as plt # matplotlib is going to be useful later

To start we will explore the principal data structure used in Iris: The Cube. We will explore how to call, write, and replace attributes, data, and coordinates contained within a Cube.